In [ ]:
using LinearAlgebra, QuadGK, Roots, FFTW
using VlasovSolvers
import VlasovSolvers: samples, Particles, PIC_step!
using Plots
using ProgressMeter

In [ ]:
function solve_PIC!(nsteps, dt, particles, meshx; plotting=false::Bool)
    potential = []
    energy_elec_from_phi = []
    energy_pic = []
    energy_hamil_elec = []
    energy_hamil_cine = []
    energy_tot = []
    L = meshx.stop
    np = length(p.x)
    progression = ProgressMeter.Progress(nsteps,desc="Loop in time: ", showspeed=true)
    animation = @animate for istep = 1:nsteps # Loop over time
        if plotting
            plot(size=(500, 500), ylim=(0, .5),
                widen=false, st=:surface, camera=(-30, 30))
            histogram2d(copy(p.x), copy(p.v), bins=500, normalize=true,
                xlabel="position", ylabel="velocity")
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end

        e1, e2, e3 = PIC_step!(p, meshx, dt)
        push!(energy_hamil_elec, e1)
        push!(energy_pic, e2)
        push!(energy_elec_from_phi, e3)
        
        ProgressMeter.next!(progression)
    end when plotting
    return energy_hamil_elec, energy_pic, energy_elec_from_phi, animation
end

# Strong Landau Damping

In [ ]:
nstep = 1000
dt = 0.05
kx = 0.5
L = 2π / kx
K = 1 # paramètre de troncature du noyau
np = 100_000 # nb de particules
nx = 32   # nb de mailles pour le calcul de l energie
ϵ = 0.5 # perturbation initiale
μ = 0.0
β = 1.0

In [ ]:
dev = CPU()
mesh1 = OneDGrid(dev, nx, 0, L)

@time (x0, y0, wei) = samples(np, kx, ϵ, μ, β)
p = Particles(x0, y0, wei, np);

In [ ]:
energy_hamil_elec, energy_pic, energy_elec_from_phi, animation = solve_PIC!(nstep, dt, p, mesh1, plotting=true);

In [ ]:
t = range(0., stop=nstep*dt, length=nstep) |> collect

plot(t, log.(sqrt.(energy_pic)), label="PIC", xlabel="time")
plot!(t, log.(sqrt.(max.(energy_hamil_elec, 10^-12))), label="Hamiltonian")
plot!(t, log.(sqrt.(energy_elec_from_phi)), label="Potential")
plot!(x-> -0.286x + 1, label="y = -0.286x + 1", 0, 20)
plot!(x->0.086671x - 3.8, label="y = 0.086671x - 3.8")
plot!(minorgrid=true, legend=:bottomleft)

In [ ]:
if animation != nothing
    gif(animation)
end